In [1]:
import torch
import numpy as np
import pickle
import cv2
import os

from torchvision.models import resnet152
from torchvision import transforms
from torch import nn

from vocab import Vocab, load_vocab
from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import MultiLayerFCReLUClassifier, PosterOnlyModel, Model
from itertools import chain
from train import train_epoches

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
POSTERS = load_data("../local/posters.npy")

RES152 = resnet152(pretrained=True)

# Freeze base layers
for child in list(RES152.children())[:-3]:
    for param in child.parameters():
        param.requires_grad = False

# Conv4, turn off grad in first 2 blocks
for child in list(list(RES152.children())[-3].children())[:2]:
    for param in child.parameters():
        param.requires_grad = False

In [18]:
CNN = nn.Sequential(*list(RES152.children())[:-1]).cuda()
classifier = MultiLayerFCReLUClassifier(dims=[1024,512,128], num_class=19, encoding_size=2048, cuda=True)

model = PosterOnlyModel(CNN, classifier, POSTERS, GENRES)

In [39]:
adam = torch.optim.Adam(filter(lambda p:p.requires_grad, model.parameters()))

sgd = torch.optim.SGD(params=filter(lambda p:p.requires_grad, model.parameters()),
                            lr=1e-3, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(sgd, 'max')

optimizer = adam
scheduler = None

loss = torch.nn.BCEWithLogitsLoss().cuda()

In [ ]:
loss_hist = []
for i in range(10):
    train_epoches(n_epochs=20, model=tomodel, train=train, loss=loss, val=val,
                  batch_size=32, optimizer=optimizer, scheduler=scheduler)
    loss_hist.append(epoch_losses)
    bn = (i+1)*50
    torch.save(tomodel.encoder, "saved/poster_encoder_{}.pth".format(bn))
    torch.save(tomodel.classifier, "saved/cnn_cls_{}.pth".format(bn))